In [1]:
import pandas as pd
import nltk
import numpy as np

#load inthe NTLK stopwords to remove articles, preposition and other words that are not actionable
from nltk.corpus import stopwords
# This allows to create individual objects from a bog of words
from nltk.tokenize import word_tokenize, sent_tokenize
# Lemmatizer helps to reduce words to the base form
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# df_eda=pd.read_csv('eda_np.txt', sep="\t", header=0)

In [3]:
# df_eda

In [4]:
df=pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/jasmineli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def process_sentence(sentence):
    new_tokens = word_tokenize(sentence)
    new_tokens = [t.lower() for t in new_tokens]
    new_tokens =[t for t in new_tokens if t not in stopwords.words('english')]
    new_tokens = [t for t in new_tokens if t.isalpha()]
    lemmatizer = WordNetLemmatizer()
    new_tokens =[lemmatizer.lemmatize(t) for t in new_tokens]
    # return " ".join(new_tokens), len(new_tokens)
    return " ".join(new_tokens)

In [6]:
# df['tags_pre'] = df['tags'].apply(lambda x: x.replace(',', ' '))

In [ ]:
titles = df['title_orig'].tolist()
# results = [process_sentence(t) for t in titles]
tokens = [process_sentence(t) for t in titles]
# tokens, length = zip(*results)
df['title_pre'] = tokens
# df['title_len'] = df['title_pre'].str.split(" ").str.len()

In [ ]:
# df[['title_len', 'units_sold']].sort_values(by='units_sold', ascending=False).head(10).mean(axis=0)

In [ ]:
# df[['title_len', 'units_sold']].sort_values(by='units_sold', ascending=False).tail(10).mean(axis=0)

In [ ]:
# tags = df['tags_pre'].tolist()
# results = [process_sentence(t) for t in tags]
# tokens, length = zip(*results)
# df['tags_pre'] = tokens
# df['tags_num'] = length

In [ ]:
# df[['tags_num', 'units_sold']].sort_values(by='units_sold', ascending=False).head(30).mean(axis=0)

In [ ]:
# df[['tags_num', 'units_sold']].sort_values(by='units_sold', ascending=False).tail(30).mean(axis=0)

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['title_pre'])

In [ ]:
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
tfidf = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
tfidf

In [ ]:
# vectorizer = TfidfVectorizer()
# vectors = vectorizer.fit_transform(df['tags_pre'])
# feature_names = vectorizer.get_feature_names()
# dense = vectors.todense()
# denselist = dense.tolist()
# tfidf_tag = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
# product color
def main_color(s):
    main_color = {"red":"red", "white":"white", "pink":"pink", "yellow":"yellow", "green":"green", "blue":"blue", "wine":"red", "burgundy":"red", "black":"black", "navy":"navy", "orange":"orange", 
    "rose":"pink", "gray":"gray", "grey":"gray", "purple":"purple", "violet":"purple", "army":"green", "leopard":"orange", "ivory":"white", 
    "brown":"brown", "coffee":"brown", "camel":"beige", "tan":"brown", "nude":"beige", "khaki":"khaki", "apricot":"yellow", "camouflage":"green", "jasper":"red"}  # ordered by importance
    for key, value in main_color.items():
        if key in s:
            return value
    return "others"
product_color = df["product_color"]
product_color = [s.lower() if type(s) is str else 'nan' for s in product_color]
product_color = [main_color(s) for s in product_color]
from matplotlib import colors
product_color = [(-0.1,-0.1,-0.1,-0.1) if s == "others" else colors.to_rgba(s) for s in product_color]

df['product_color_rgb'] = [np.array(t) for t in product_color]

# log prices
df['log_price'] = [np.log(p) for p in df["price"]]
df['log_retail_price'] = [np.log(p) for p in df["retail_price"]]

# log merchant rating count
df['log_merchant_rating_count'] = np.log(df['merchant_rating_count'])

# urgent text
df['urgent'] = [1 if s == "Quantité limitée !" else 0 for s in df["urgency_text"]]

In [ ]:
data = df[["log_price", "log_retail_price", "uses_ad_boosts", "badges_count", "badge_local_product", 
           "badge_product_quality", "badge_fast_shipping", "urgent", "units_sold"]]
df2 = pd.concat([data, tfidf], axis=1)
label = [1 if sales > 100 else 0 for sales in data["units_sold"]]
df2['high_sale'] = label
print(df2['high_sale'].value_counts())

In [ ]:
rgb = df["product_color_rgb"]
rgb = np.stack(rgb.values, axis=0)
for i in range(4):
    df2["product_color_rgb"+str(i)] = rgb[:,i]
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
print(df2.shape)

In [ ]:
X = df2.loc[:, ~df2.columns.isin(['high_sale', 'units_sold'])]
# X = tfidf
y = df2['high_sale']

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.layers import Input, Dense, Concatenate
from keras.models import Model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.14286, random_state=42)

In [ ]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of dev examples = " + str(X_dev.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(y_dev.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

In [ ]:
X_train = tf.expand_dims(X_train, axis=-1)
X_dev = tf.expand_dims(X_dev, axis=-1)
input_shape = X_train.shape[1:]

In [ ]:
print(input_shape)

In [ ]:
# from sklearn.linear_model import LogisticRegression

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# score = classifier.score(evals_X, evals_y)

# print("Accuracy:", score)

In [ ]:
from keras.layers import AveragePooling1D, PReLU, LeakyReLU

In [ ]:
# define model
model = Sequential()
model.add(Conv1D(filters=24, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=30)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=1000,
    verbose=2,
    validation_data=(X_dev, y_dev),
    batch_size=16,
    callbacks=[es]
)
model.summary()